In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import csv
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Variables

1.   all_files = Files path containing all text files
2.   CSV_path = Folder path for CSV file outputs with numerical filter
3.   xlsx_path = Folder path for Excel file outputs with numerical filter
4.   file_name = Filename without the extension and folder path
5.   CSV_File = CSV file in CSV Path




Getting all text files from sample folder

In [ ]:
all_files = glob.glob("/content/drive/MyDrive/Data/Analyst report sample/*.txt")

In [ ]:
all_files

['/content/drive/MyDrive/Data/Analyst report sample/MS_05262017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/CHTR_05302017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/LM_06132017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/SNV_06272017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/VRSN_06292017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/VRSK_07212017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/POT_07312017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/ALB_08172017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/AZO_08242017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/ADSK_09012017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/ZION_09152017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/DOX_09222017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/IVZ_10042017.txt',
 '/content/drive/MyDrive/Data/Analyst report sample/ADSK_10052017.txt',
 '

In [ ]:
type(all_files)

list

Defining CSV and Excel Folder for outputs

In [ ]:
CSV_path = "/content/drive/Shareddrives/Msc DS-2020 /Fin Num Claim Detection/Data/CSV_Test_Path"

In [ ]:
XLSX_path = "/content/drive/Shareddrives/Msc DS-2020 /Fin Num Claim Detection/Data/XLSX_complete"

Extracting filename without the extension

In [ ]:
def Extract_filename(Text_data_path):
    eg = os.path.basename(Text_data_path)
    file_name, f_ext = os.path.splitext(eg)
    return file_name

Filtering numerical Sentences from txt files and converting it into CSV output files.

In [ ]:
for Text_data_path in all_files:

    data = open(Text_data_path, encoding = "latin1")
    list_data = [line.rstrip('\n') for line in data]
    str_data = str(list_data)
    sentences = sent_tokenize(str_data)
    text = []

    for eachLine in sentences:
        
        if any(char.isdigit() for char in eachLine):
            print (eachLine)
            text.append(eachLine)
            
    file_name = Extract_filename(Text_data_path)
    
    df = pd.DataFrame(data={"text":text})
    df.to_excel(XLSX_path+"/"+file_name+".xlsx",index=False)
    
    print("##################################################################################################################################################################################################################################################################")


Storing CSV Filenames in a list

In [ ]:
XLSX_File_List = []

In [ ]:
path = XLSX_path
for XLSX_File in glob.glob(os.path.join(path, '*.xlsx')):
    print(XLSX_File)
    XLSX_File_List.append(XLSX_File)
    df = pd.read_excel(XLSX_File)
    print(df.shape)

In [ ]:
XLSX_File_List

In [ ]:
df.shape

(38, 1)

Snorkel

In [ ]:
pip install snorkel

In [ ]:
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(df, test_size=0.2, random_state=25)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 30
No. of testing examples: 8


In [ ]:
from snorkel.labeling import labeling_function

In [ ]:
inclaim=1
outclaim=0
abstain=-1

In [ ]:
@labeling_function()
def lf_contains_growth(df):
  return inclaim if "growth" in df.text.lower() else outclaim

@labeling_function()
def lf_contains_exceeds(df):
    return inclaim if "exceeds" in df.text.lower() else outclaim

@labeling_function()
def lf_contains_gains(df):
  return inclaim if "increased" in df.text.lower() else outclaim

In [ ]:
from snorkel.labeling import PandasLFApplier

lfs = [lf_contains_growth, lf_contains_exceeds,lf_contains_gains]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=training_data)

100%|██████████| 30/30 [00:00<00:00, 11415.14it/s]


In [ ]:
L_train

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [ ]:
coverage_grow, coverage_exceed, coverage_gains = (L_train != outclaim).mean(axis=0)
print(f"check_out coverage: {coverage_grow * 100:.1f}%")
print(f"coverage_exceed: {coverage_exceed * 100:.1f}%")
print(f"coverage_gains: {coverage_gains * 100:.1f}%")

check_out coverage: 6.7%
coverage_exceed: 0.0%
coverage_gains: 3.3%
